In [77]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [78]:
df = pd.read_csv('capital_bike_data/train.csv')

In [79]:
def feature_engin(df):
    
    mms = MinMaxScaler()
    
    df['date'] = pd.to_datetime(df['datetime'])
    df.set_index(['date'], inplace=True)
    df['month'] = df.index.month
    df['hour'] = df.index.hour
    df[['atemp','windspeed', 'humidity']] = mms.fit_transform(df[['atemp','windspeed', 'humidity']])
    df = pd.get_dummies(df, columns=['weather'], drop_first=True)
    df = pd.get_dummies(df, columns=['season'], drop_first=True)
    df.drop(['datetime'], axis=1, inplace=True)
    df.reset_index(inplace=True, drop=True)

## STEP 2. use Regression model

DONE:
- get a first impression & check out data
- find corrolations (heatmap & plotting)
- feature engineer

create your own features by binning data


## STEP 2.1. Split into test & train data

In [80]:
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [81]:
feature_engin(df)

/home/pepper/Applications/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [84]:
df.head(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,month,hour
date,,,,,,,,,,,,,,
2011-01-01 00:00:00,2011-01-01 00:00:00,1,0,0,1,9.84,0.305068,0.81,0.0,3,13,16,1,0
2011-01-01 01:00:00,2011-01-01 01:00:00,1,0,0,1,9.02,0.288064,0.80,0.0,8,32,40,1,1
2011-01-01 02:00:00,2011-01-01 02:00:00,1,0,0,1,9.02,0.288064,0.80,0.0,5,27,32,1,2


In [85]:
df = pd.get_dummies(df, columns=['weather'], drop_first=True)
df = pd.get_dummies(df, columns=['season'], drop_first=True)
df.drop(['datetime'], axis=1, inplace=True)
df.reset_index(inplace=True, drop=True)
#df[['atemp','windspeed', 'humidity']] = mms.fit_transform(df[['atemp','windspeed', 'humidity']])

In [86]:
df.head()

,holiday,workingday,temp,atemp,humidity,windspeed,casual,registered,count,month,hour,weather_2,weather_3,weather_4,season_2,season_3,season_4
0,0,0,9.84,0.305068,0.81,0.0,3,13,16,1,0,0,0,0,0,0,0
1,0,0,9.02,0.288064,0.80,0.0,8,32,40,1,1,0,0,0,0,0,0
2,0,0,9.02,0.288064,0.80,0.0,5,27,32,1,2,0,0,0,0,0,0
3,0,0,9.84,0.305068,0.75,0.0,3,10,13,1,3,0,0,0,0,0,0
4,0,0,9.84,0.305068,0.75,0.0,0,1,1,1,4,0,0,0,0,0,0


In [87]:
y = df['count'] #vector

In [88]:
df = df.drop(['count', 'casual', 'registered', 'temp'], axis=1)
X_features = df.columns
X = df[X_features] #matrix

In [89]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [91]:
m = LinearRegression()
m_forest = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)

In [92]:
m.fit(X_train, y_train)
m_forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

## STEP 2.2 Prediction

In [93]:
ypred = m.predict(X_test)
ypred_forest = m_forest.predict(X_test)

In [94]:
ypred

array([283.90076816,  -6.08916615, 230.67206234, ..., 137.50460939,
       180.46799103, 113.58145887])

In [95]:
ypred_forest

array([337.2897689 ,  77.95577318, 204.119938  , ...,  24.87304935,
       337.2897689 , 212.07022991])

In [96]:
same = ypred == ypred_forest
same.sum()

0

In [97]:
len(X_test)

2722

## STEP 2.3 Score & Cross-Validation

In [98]:
# for random forest model

In [99]:
cvs = cross_val_score(m_forest, X_train, y_train, cv=10)
cvs.mean()

0.4139592327026046

In [100]:
cvs.std()

0.022636943204849092

In [101]:
m_forest.score(X_train, y_train)

0.416941426897915

In [102]:
# for linear regression model

In [103]:
# remember this scores does not work well with Regression Models so we use RMSLE

In [104]:
# Squareroot does not work with negativ numbers, so ...

In [105]:
ypred.min(), ypred.max(), y_test.min(), y_test.max(), 

(-128.32713151614513, 431.1617401981484, 1, 977)

In [106]:
ypred[ypred <= 0] = 10

In [107]:
ypred.min(), ypred.max()

(0.11284612145817974, 431.1617401981484)

In [108]:
def rmsle(p,a):
    return np.sqrt(mean_squared_log_error([p],[a]))

In [112]:
rmsle(y_test, ypred), rmsle(y_test, ypred_forest)

(1.1777750772110325, 0.9212023158447624)

## STEP 3 Prediction w. test data

In [113]:
df_t = pd.read_csv('capital_bike_data/test.csv')

In [114]:
df_t.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [115]:
feature_engin(df_t)
df_t.head()

/home/pepper/Applications/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,month,hour
date,,,,,,,,,,,
2011-01-20 00:00:00,2011-01-20 00:00:00,1,0,1,1,10.66,0.2273,0.47619,0.464346,1,0
2011-01-20 01:00:00,2011-01-20 01:00:00,1,0,1,1,10.66,0.2727,0.47619,0.000000,1,1
2011-01-20 02:00:00,2011-01-20 02:00:00,1,0,1,1,10.66,0.2727,0.47619,0.000000,1,2
2011-01-20 03:00:00,2011-01-20 03:00:00,1,0,1,1,10.66,0.2576,0.47619,0.196458,1,3
2011-01-20 04:00:00,2011-01-20 04:00:00,1,0,1,1,10.66,0.2576,0.47619,0.196458,1,4


In [116]:
df_t = pd.get_dummies(df_t, columns=['weather'], drop_first=True)
df_t = pd.get_dummies(df_t, columns=['season'], drop_first=True)
df_t.drop(['datetime', 'temp'], axis=1, inplace=True)
df_t.reset_index(inplace=True, drop=True)

In [121]:
df_t.head()

,holiday,workingday,atemp,humidity,windspeed,month,hour,weather_2,weather_3,weather_4,season_2,season_3,season_4
0,0,1,0.2273,0.47619,0.464346,1,0,0,0,0,0,0,0
1,0,1,0.2727,0.47619,0.000000,1,1,0,0,0,0,0,0
2,0,1,0.2727,0.47619,0.000000,1,2,0,0,0,0,0,0
3,0,1,0.2576,0.47619,0.196458,1,3,0,0,0,0,0,0
4,0,1,0.2576,0.47619,0.196458,1,4,0,0,0,0,0,0


In [122]:
ypred_t_forest = m_forest.predict(df_t)

In [123]:
ypred_t = m.predict(df_t)

In [124]:
ypred_t[ypred_t<=0]=15
ypred_t_forest[ypred_t_forest<=0]=15

In [125]:
ypred_t.min(), ypred_t_forest.min()

(0.14507424916244815, 24.873049353606433)

## STEP 4: submission & online verification 

In [126]:
sub_df = pd.read_csv('capital_bike_data/sampleSubmission.csv')
df_t = pd.read_csv('capital_bike_data/test.csv')

In [127]:
len(sub_df), len(df_t)

(6493, 6493)

In [128]:
ypred_t = np.round(ypred_t, decimals=0)
ypred_t_forest = np.round(ypred_t_forest, decimals=0)

In [129]:
# for i in ypred_t np.int(i)

In [130]:
sub_df['count'] = ypred_t

In [131]:
#sub_df['count'][sub_df['count']==0.0 = 5  

In [132]:
sub_df['count'] = sub_df['count'].astype(int)
sub_df.head()

,datetime,count
0,2011-01-20 00:00:00,7
1,2011-01-20 01:00:00,20
2,2011-01-20 02:00:00,27
3,2011-01-20 03:00:00,34
4,2011-01-20 04:00:00,42


In [133]:
sub_df.reset_index(drop=True, inplace=True)

In [134]:
sub_df.head()

,datetime,count
0,2011-01-20 00:00:00,7
1,2011-01-20 01:00:00,20
2,2011-01-20 02:00:00,27
3,2011-01-20 03:00:00,34
4,2011-01-20 04:00:00,42


In [135]:
sub_df.to_csv('kaggel_submission_lin.csv', index=False)

In [136]:
sub_df['count'] = ypred_t_forest
sub_df['count'] = sub_df['count'].astype(int)

In [137]:
sub_df.to_csv('kaggel_submission_forest.csv', index=False)

In [138]:
check = pd.read_csv('kaggel_submission_forest.csv')
check.head(30)

,datetime,count
0,2011-01-20 00:00:00,25
1,2011-01-20 01:00:00,25
2,2011-01-20 02:00:00,25
3,2011-01-20 03:00:00,25
4,2011-01-20 04:00:00,25
5,2011-01-20 05:00:00,25
6,2011-01-20 06:00:00,78
7,2011-01-20 07:00:00,204
8,2011-01-20 08:00:00,204
9,2011-01-20 09:00:00,204


In [139]:
check2 = pd.read_csv('kaggel_submission_lin.csv')
check2.head(30)

,datetime,count
0,2011-01-20 00:00:00,7
1,2011-01-20 01:00:00,20
2,2011-01-20 02:00:00,27
3,2011-01-20 03:00:00,34
4,2011-01-20 04:00:00,42
5,2011-01-20 05:00:00,30
6,2011-01-20 06:00:00,32
7,2011-01-20 07:00:00,53
8,2011-01-20 08:00:00,62
9,2011-01-20 09:00:00,91
